# Programming Test Conference room scheduling. 
### Find the nearest open conference room for a team in which a team can hold its meeting. 

Given n team members with the floor on which they work and the time
they want to meet, and a list of conference rooms identified by their floor and room number as a decimal
number, maximum number of people it fits and pairs of times they are open - find the best place for the
team to have their meeting. If there is more than one room available that fits the team at the chosen time
then the best place is on the floor the closest to where the team works. 


```E.g. rooms.txt
7.11,8,9:00,9:15,14:30,15:00 8.23,6,10:00,11:00,14:00,15:00 8.43,7,11:30,12:30,17:00,17:30
9.511,9,9:30,10:30,12:00,12:15,15:15,16:15 9.527,4,9:00,11:00,14:00,16:00
9.547,8,10:30,11:30,13:30,15:30,16:30,17:30 
```

__Input__: 5,8,10:30,11:30 # 5 team members, located on the
8th floor, meeting time 10:30 - 11:30 

__Output__: 9.547

In [48]:
import os.path
from os import path
closingTime = '18:00' # Assuming the meeting rooms are available till 6 PM in evening.

class time: 
    def __init__(self, arg):
        self.hour = int(arg.split(":")[0])
        self.min = int(arg.split(":")[1])

    def __eq__(self, other):
        return self.hour == other.hour and self.min == other.min
    
    def __str__(self):
        return "{}:{}".format(self.hour, self.min)
    
    def __ge__(self, other):
        return self.hour>other.hour or (self.hour==other.hour and self.min>=other.min)

    def __le__(self, other):
        return self.hour<other.hour or (self.hour==other.hour and self.min<=other.min)

    def add_duration(self, mins):
        self.hour += int(mins/60)
        self.min  += mins%60

    def mins_diff(self, other):
        return abs(self.hour-other.hour)*60+abs(self.min-other.min)
    
    
def sample_loader():
    '''Form a file if not present and read from the file...
        Create a file..''' 
    default_room = "7.11,8,9:00,9:15,14:30,15:00 8.23,6,10:00,11:00,14:00,15:00 8.43,7,11:30,12:30,17:00,17:30 9.511,9,9:30,10:30,12:00,12:15,15:15,16:15 9.527,4,9:00,11:00,14:00,16:00 9.547,8,10:30,11:30,13:30,15:30,16:30,17:30"
    default_file = "room.txt"
    if not path.exists(default_file):
        fp = open(default_file, "w")
        fp.write(default_room)
        fp.close()

    fp = open(default_file, "r")
    room = fp.read()
    return room.split(" ") # Split to see the all rooms that are available...

In [49]:
def create_dataset():
    ''' Form a list of tuples.
        (Room Capacity, Room Floor, Start time of Availability, Duration of availability, room number) '''
    dic = []
    for room in sample_loader():
        room = room.split(',')
        roomNo = room[0] 
        roomCapacity = int(room[1]) 
        roomTimes = room[2:]
        if len(roomTimes)!=0:
            roomTimes.append(closingTime)
            for ind in range(len(roomTimes)-1):
                start = time(roomTimes[ind])
                duration = start.mins_diff(time(roomTimes[ind+1]))
                dic.append((roomCapacity, int(roomNo.split('.')[0]), start, duration, roomNo))
    return dic

In [42]:
# We can prioratize the meeting with capacity first, floor, Time, Duration...
inp = "5,8,10:30,11:30"
def find_the_meeting_room(inp):
    inp_list = inp.split(',')
    capacity = int(inp_list[0])
    floor = int(inp_list[1])
    start = time(inp_list[2])
    duration = start.mins_diff(time(inp_list[3]))
    # First we check for the capacity of the room, if capacity do not match we do not consider them for other calculations.
    # Second we remove all the time which do not meet our criteria.. 
    dic = create_dataset()
    dic = list(filter(lambda x:x[0] >= capacity and x[2]==start and x[3]>=duration, dic))
    print("Room nearest to the Input is : ", sorted(dic, key = lambda x: (abs(x[1]-floor), abs(x[0]-capacity)))[0][-1])
find_the_meeting_room(inp)    

Room nearest to the Input is :  9.547


# Please explain: 
### how you solved the problem and how it would behave based on the different parameters __(number of team members, longer meeting times, many rooms with random booking times).__

The solution to the problem lies on following assumptions:
    * We are not only reserving rooms with the capacity to hold all it's memebers.
    * The meeting rooms can not be booked after 6 PM. 

We first make tuple of different time and duration for which the room is avaialble. 
Structure of the tuple is as follow:

``` tuple = (Room Capacity, Room Floor, Start time of Availability, Duration of availability, room number) ```

__Please note__ The order of tuple is selected at random. 

With the list of such tuples we can iterate over the list to:
    
    > Filter all the meeting places which has less seats than required at input.
    > Also filter the meeting places which do not match with start time of meeting.
    > Then sort the remaining tuple with the first with the closest room and then based on the capacity. 
    > The room with the minimun distance and exact quantity of seats is returned by the program. 

## How would you test the program to ensure it always produced the correct results?
Feel Free to fill here !!

## Bonus Question.. 

For extra credit, can you improve the solution to split the meeting across more than one room if say only
one room is available for a fraction of the meeting and another room is free later to hold the remainder of
the meeting during the set time. If you want to make this more powerful - assume that the number of room
splits can happen in proportion to the length of the meeting so that say if a meeting is 8 hrs long then the
algorithm could schedule it across say up to 4 rooms if a single room was not available for the whole time.

In [52]:
inp = "2,8,09:30,12:30" 
def find_the_meeting_room(inp):
    inp_list = inp.split(',')
    capacity = int(inp_list[0])
    floor = int(inp_list[1])
    start = time(inp_list[2])
    duration = start.mins_diff(time(inp_list[3]))
    # First we check for the capacity of the room, if capacity do not match we do not consider them for other calculations.
    # Second we remove all the time which do not meet our criteria.. 
    dic = create_dataset()
    dic = list(filter(lambda x:x[0] >= capacity, dic))
    while duration>0 and start<=time(closingTime):
        dic = list(filter(lambda x:x[2] >= start, dic))
        sorted_list = sorted(dic, key = lambda x: (abs(x[1]-floor), abs(x[0]-capacity)))
        #print(sorted_list)
        duration -= sorted_list[0][-2]
        start.add_duration(sorted_list[0][-2])
        #print(start, duration, sorted_list[0][-2])
find_the_meeting_room(inp)